# ETAPA 3b — Análise Triple-DiD: IA Generativa, Conectividade e Emprego Formal

**Dissertação:** Inteligência Artificial Generativa e o Mercado de Trabalho Brasileiro.

**Objetivo:** Estimar o efeito diferencial da IA generativa entre municípios de alta e baixa conectividade (Triple-DiD). Unidade: ocupação × município × mês. Coeficiente de interesse: **β₇** (triple_did = post × alta_exp × alta_conectividade).

**Input:** `data/output/painel_caged_municipio_anatel.parquet` (Notebook 3a).

### 0. Dependências

Conferir pacotes instalados e instalar o necessário para o notebook (pandas, numpy, pyfixest e dependências como platformdirs).

In [3]:
# Etapa 3b.0 — Conferir e instalar dependências
!pip install pandas numpy pyfixest platformdirs pyarrow -q
print("Pacotes principais (após install):")
!pip list

Pacotes principais (após install):
Package                            Version
---------------------------------- --------------
aiohappyeyeballs                   2.6.1
aiohttp                            3.13.2
aiosignal                          1.4.0
altair                             5.4.1
annotated-types                    0.6.0
anyio                              4.12.0
appnope                            0.1.3
argon2-cffi                        25.1.0
argon2-cffi-bindings               25.1.0
arrow                              1.3.0
asttokens                          2.4.1
async-lru                          2.0.5
attrs                              23.2.0
Authlib                            1.6.6
babel                              2.17.0
backports.tarfile                  1.2.0
basedosdados                       2.0.2
beartype                           0.22.8
beautifulsoup4                     4.12.2
bleach                             6.2.0
blinker                            1.8.2
bro

### Estratégia de identificação

| Elemento | Especificação |
|----------|---------------|
| Unidade | Ocupação (CBO 4d) × Município × Mês |
| Tratamento (1) | Alta exposição à IA (ILO score > mediana) |
| Tratamento (2) | Alta conectividade (penetração BL > mediana) |
| Evento | Lançamento ChatGPT (Nov/2022) |
| FE | cbo_4d + uf_periodo |
| Clustering | id_municipio |
| Coef. interesse | β₇ (triple_did) |

### 1. Configuração e carga de dados

Carregar painel 3a, winsorizar salários (P1/P99).

In [4]:
# Etapa 3b.1 — Configuração e carga
import warnings
import pandas as pd
import numpy as np
import pyfixest as pf
from pathlib import Path

warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", message=".*multicollinearity.*", category=UserWarning)

# Sempre usar a pasta notebook/data
def _find_project_root():
    p = Path.cwd().resolve()
    for _ in range(5):
        if (p / "notebook").is_dir():
            return p
        p = p.parent if p.parent != p else p
    return Path.cwd().resolve()
PROJECT_ROOT = _find_project_root()
DATA_ROOT = PROJECT_ROOT / "notebook" / "data"
DATA_OUTPUT = DATA_ROOT / "output"
CACHE_DIR = DATA_ROOT / "cache"
OUTPUTS_TABLES = PROJECT_ROOT / "notebook" / "outputs" / "tables"
OUTPUTS_FIGURES = PROJECT_ROOT / "notebook" / "outputs" / "figures"
USE_CACHE_3B = True  # Se True, seção 8 usa cache e checkpoints; se False, tudo em memória
for d in [OUTPUTS_TABLES, OUTPUTS_FIGURES, CACHE_DIR]:
    d.mkdir(parents=True, exist_ok=True)

PAINEL_FILE = DATA_OUTPUT / "painel_caged_municipio_anatel.parquet"
PAINEL_FILE_V2 = DATA_OUTPUT / "painel_caged_municipio_anatel_v2.parquet"
VCOV_SPEC = {"CRV1": "id_municipio"}
OUTCOMES = {"ln_salario_real_adm": "Log(Sal. Real Adm.)", "ln_admissoes": "Log(Admissões)", "pct_superior_adm": "% Superior", "idade_media_adm": "Idade Média"}

print("Data output:", DATA_OUTPUT.resolve())
if PAINEL_FILE_V2.exists():
    df = pd.read_parquet(PAINEL_FILE_V2)
    print("Painel v2 (faixa etária, fibra, capital) carregado:", PAINEL_FILE_V2.resolve())
else:
    df = pd.read_parquet(PAINEL_FILE)
    print("Painel (sem v2) carregado:", PAINEL_FILE.resolve())
for c in ["salario_medio_adm", "salario_real_adm"]:
    if c in df.columns:
        lo, hi = df[c].quantile(0.01), df[c].quantile(0.99)
        df[c] = df[c].clip(lo, hi)
if "ln_salario_real_adm" in df.columns:
    df["ln_salario_real_adm"] = np.log(df["salario_real_adm"].clip(lower=1))
print(f"Painel: {len(df):,} obs | {df['id_municipio'].nunique():,} municípios | {df['cbo_4d'].nunique()} ocupações")

Data output: /Users/manebrasil/Documents/Projects/Dissetação Mestrado/notebook/data/output
Painel v2 (faixa etária, fibra, capital) carregado: /Users/manebrasil/Documents/Projects/Dissetação Mestrado/notebook/data/output/painel_caged_municipio_anatel_v2.parquet
Painel: 5,534,808 obs | 657 municípios | 614 ocupações


### 2. Tabela de balanço por conectividade (pré-tratamento)

In [5]:
# Etapa 3b.2 — Balanço
pre = df[df["post"] == 0]
print(pre.groupby("alta_conectividade").agg(
    n_obs=("cbo_4d", "count"),
    admissoes_media=("admissoes", "mean"),
    salario_medio=("salario_medio_adm", "mean"),
    penetracao_media=("penetracao_bl", "mean"),
).round(2))

                      n_obs  admissoes_media  salario_medio  penetracao_media
alta_conectividade                                                           
0                    205077             5.02        1330.58              0.18
1                   2126257            15.02        1742.66              0.69


### 3. DiD por subgrupo de conectividade (motivação)

Estimar DiD (post × alta_exp) separadamente para municípios de alta e baixa conectividade.

In [6]:
# Etapa 3b.3 — DiD por subgrupo
df["post_alta_exp"] = df["post"] * df["alta_exp"]
for conn_label, mask in [("Alta conect.", df["alta_conectividade"] == 1), ("Baixa conect.", df["alta_conectividade"] == 0)]:
    d = df.loc[mask].dropna(subset=["ln_salario_real_adm"])
    m = pf.feols("ln_salario_real_adm ~ post_alta_exp | cbo_4d + uf_periodo", data=d, vcov=VCOV_SPEC)
    c = m.coef().get("post_alta_exp", m.coef().iloc[0])
    print(f"{conn_label}: coef = {float(c):.4f}, se = {float(m.se().loc['post_alta_exp']):.4f}")

Alta conect.: coef = -0.0058, se = 0.0018
Baixa conect.: coef = -0.0089, se = 0.0046


### 4. Triple-DiD — Modelo principal

outcome ~ triple_did + post_alta_exp + post_alta_conect + alta_exp_alta_conect | cbo_4d + uf_periodo

In [7]:
# Etapa 3b.4 — Triple-DiD principal
formula = "ln_salario_real_adm ~ triple_did + post_alta_exp + post_alta_conect + alta_exp_alta_conect | cbo_4d + uf_periodo"
m = pf.feols(formula, data=df.dropna(subset=["ln_salario_real_adm"]), vcov=VCOV_SPEC)
print(m.summary())
results_triple = pd.DataFrame({"coef": m.coef(), "se": m.se(), "p_value": m.pvalue()})
results_triple.to_csv(OUTPUTS_TABLES / "triple_did_main_etapa3b.csv")

###

Estimation:  OLS
Dep. var.: ln_salario_real_adm, Fixed effects: cbo_4d+uf_periodo
Inference:  CRV1
Observations:  5534808

| Coefficient          |   Estimate |   Std. Error |   t value |   Pr(>|t|) |   2.5% |   97.5% |
|:---------------------|-----------:|-------------:|----------:|-----------:|-------:|--------:|
| triple_did           |     -0.016 |        0.008 |    -1.888 |      0.059 | -0.032 |   0.001 |
| post_alta_exp        |      0.008 |        0.008 |     1.069 |      0.285 | -0.007 |   0.024 |
| post_alta_conect     |      0.017 |        0.008 |     2.239 |      0.025 |  0.002 |   0.032 |
| alta_exp_alta_conect |      0.037 |        0.010 |     3.743 |      0.000 |  0.017 |   0.056 |
---
RMSE: 0.555 R2: 0.969 R2 Within: 0.0 
None


### 5. Event study por grupo de conectividade

Dummies (tempo_relativo × alta_exp) para alta e baixa conectividade; referência t = -1.

In [8]:
# Etapa 3b.5 — Event study por conectividade
# Dummies (tempo_relativo × alta_exp) para alta e baixa conectividade; referência t = -1.
# Nomes das variáveis: sem "-" na fórmula (formulaic interpreta "did_t-12" como did_t menos 12).
def _nome_did(t):
    return f"did_m{-t}" if t < 0 else f"did_{t}"

BIN_MIN, BIN_MAX, REF = -12, 24, -1
df["t_bin"] = df["tempo_relativo_meses"].clip(lower=BIN_MIN, upper=BIN_MAX)
for conn_val, label in [(1, "Alta"), (0, "Baixa")]:
    d = df[(df["alta_conectividade"] == conn_val) & df["ln_salario_real_adm"].notna()].copy()
    ts = [t for t in sorted(d["t_bin"].unique()) if t != REF]
    did_vars = [_nome_did(t) for t in ts]
    for t, v in zip(ts, did_vars):
        d[v] = ((d["t_bin"] == t) & (d["alta_exp"] == 1)).astype(int)
    if did_vars:
        m = pf.feols(f"ln_salario_real_adm ~ {' + '.join(did_vars)} | cbo_4d + uf_periodo", data=d, vcov=VCOV_SPEC)
        print(f"{label} conect.: {len(ts)} coeficientes estimados")

Alta conect.: 36 coeficientes estimados
Baixa conect.: 36 coeficientes estimados


### 6. Testes de robustez e síntese

Placebo temporal (Dez/2021); cutoff Q75. Correção FDR para múltiplos outcomes (opcional). Síntese: comparar com Etapa 2 — efeito concentrado onde a adoção de IA é viável.

In [9]:
# Etapa 3b.6 — Robustez (placebo) e múltiplos outcomes
# Placebo: post = 1 a partir de Dez/2021
df["post_placebo"] = ((df["ano"] == 2021) & (df["mes"] >= 12)) | (df["ano"] > 2021)
df["triple_did_placebo"] = df["post_placebo"].astype(int) * df["alta_exp"] * df["alta_conectividade"]
df["post_alta_exp_placebo"] = df["post_placebo"].astype(int) * df["alta_exp"]
m_placebo = pf.feols("ln_salario_real_adm ~ triple_did_placebo + post_alta_exp_placebo | cbo_4d + uf_periodo",
                     data=df.dropna(subset=["ln_salario_real_adm"]), vcov=VCOV_SPEC)
print("Placebo Dez/2021 (triple_did_placebo):", m_placebo.coef().get("triple_did_placebo"), "p =", m_placebo.pvalue().get("triple_did_placebo"))
print("Esperado: não significativo.")
if USE_CACHE_3B:
    import json
    with open(CACHE_DIR / "placebo_secao6.json", "w") as f:
        json.dump({"triple_did_placebo": {"coef": float(m_placebo.coef().get("triple_did_placebo")), "se": float(m_placebo.se().get("triple_did_placebo")), "pval": float(m_placebo.pvalue().get("triple_did_placebo"))}}, f, indent=2)

Placebo Dez/2021 (triple_did_placebo): 0.03653039627709792 p = 5.672064404560473e-05
Esperado: não significativo.


### 7. Correção de tendência diferencial pré (Caminho 1)

Se o placebo falha por tendência pré-existente no grupo alta_exp × alta_conectividade, controlar por **alta_exp × alta_conectividade × trend** (trend = 1, 2, …, T). Se β₇ (triple_did) permanece significativo após incluir a tendência, o efeito é robusto; se não, o efeito era explicado pela tendência pré.

In [10]:
# Etapa 3b.7 — Tendência diferencial pré (Caminho 1)
periodos_ordenados = sorted(df["periodo"].unique())
trend_map = {p: i + 1 for i, p in enumerate(periodos_ordenados)}
df["trend"] = df["periodo"].map(trend_map)
df["trend_exp_conect"] = df["alta_exp"] * df["alta_conectividade"] * df["trend"]
df["trend_exp"] = df["alta_exp"] * df["trend"]
df["trend_conect"] = df["alta_conectividade"] * df["trend"]

outcome = "ln_salario_real_adm"
formula_trend = (
    f"{outcome} ~ triple_did + post_alta_exp + post_alta_conect + alta_exp_alta_conect"
    " + trend_exp_conect + trend_exp + trend_conect | cbo_4d + uf_periodo"
)
model_trend = pf.feols(formula_trend, data=df.dropna(subset=[outcome]), vcov=VCOV_SPEC)
print("Modelo com tendência diferencial pré:")
print(model_trend.summary())

# Comparar com modelo original (seção 4)
formula_orig = "ln_salario_real_adm ~ triple_did + post_alta_exp + post_alta_conect + alta_exp_alta_conect | cbo_4d + uf_periodo"
model_orig = pf.feols(formula_orig, data=df.dropna(subset=["ln_salario_real_adm"]), vcov=VCOV_SPEC)
print("\nComparação: Original vs. com tendência")
pf.etable([model_orig, model_trend])

Modelo com tendência diferencial pré:
###

Estimation:  OLS
Dep. var.: ln_salario_real_adm, Fixed effects: cbo_4d+uf_periodo
Inference:  CRV1
Observations:  5534808

| Coefficient          |   Estimate |   Std. Error |   t value |   Pr(>|t|) |   2.5% |   97.5% |
|:---------------------|-----------:|-------------:|----------:|-----------:|-------:|--------:|
| triple_did           |      0.006 |        0.010 |     0.668 |      0.504 | -0.012 |   0.025 |
| post_alta_exp        |     -0.005 |        0.009 |    -0.596 |      0.552 | -0.023 |   0.012 |
| post_alta_conect     |     -0.015 |        0.007 |    -2.070 |      0.039 | -0.030 |  -0.001 |
| alta_exp_alta_conect |      0.034 |        0.010 |     3.375 |      0.001 |  0.014 |   0.054 |
| trend_exp_conect     |     -0.001 |        0.000 |    -1.321 |      0.187 | -0.001 |   0.000 |
| trend_exp            |      0.000 |        0.000 |     0.616 |      0.538 | -0.000 |   0.001 |
| trend_conect         |      0.001 |        0.000 |     2

GT(_tbl_data=   level_0               level_1                      0                      1
0     coef            triple_did    -0.016 <br> (0.008)     0.006 <br> (0.010)
1     coef         post_alta_exp     0.008 <br> (0.008)    -0.005 <br> (0.009)
2     coef      post_alta_conect    0.017* <br> (0.008)   -0.015* <br> (0.007)
3     coef  alta_exp_alta_conect  0.037*** <br> (0.010)  0.034*** <br> (0.010)
4     coef      trend_exp_conect                           -0.001 <br> (0.000)
5     coef             trend_exp                            0.000 <br> (0.000)
6     coef          trend_conect                           0.001* <br> (0.000)
7       fe            uf_periodo                      x                      x
8       fe                cbo_4d                      x                      x
9    stats          Observations                5534808                5534808
10   stats             S.E. type       by: id_municipio       by: id_municipio
11   stats         R<sup>2</sup>                  0.969                  0.969
12   stats  R<sup>2</sup> Within                  0.000                  0.000, _body=<great_tables._gt_data.Body object at 0x12899d300>, _boxhead=Boxhead([ColInfo(var='level_0', type=<ColInfoTypeEnum.row_group: 3>, column_label='level_0', column_align='center', column_width=None), ColInfo(var='level_1', type=<ColInfoTypeEnum.stub: 2>, column_label='level_1', column_align='center', column_width=None), ColInfo(var='0', type=<ColInfoTypeEnum.default: 1>, column_label='(1)', column_align='center', column_width=None), ColInfo(var='1', type=<ColInfoTypeEnum.default: 1>, column_label='(2)', column_align='center', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x127ac5e40>, _spanners=Spanners([SpannerInfo(spanner_id='ln_salario_real_adm', spanner_level=1, spanner_label='ln_salario_real_adm', spanner_units=None, spanner_pattern=None, vars=['0', '1'], built=None)]), _heading=Heading(title=None, subtitle=None, preheader=None), _stubhead=None, _summary_rows=<great_tables._gt_data.SummaryRows object at 0x11f24ea10>, _summary_rows_grand=<great_tables._gt_data.SummaryRows object at 0x11f24f130>, _source_notes=['Significance levels: * p < 0.05, ** p < 0.01, *** p < 0.001. Format of coefficient cell:\nCoefficient \n (Std. Error)'], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x127ac4cd0>, _formats=[], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_layout=OptionsInfo(scss=True, category='table', type='value', value='fixed'), table_margin_left=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_margin_right=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_background_color=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_additional_css=OptionsInfo(scss=False, category='table', type='values', value=[]), table_font_names=OptionsInfo(scss=False, category='table', type='values', value=['-apple-system', 'BlinkMacSystemFont', 'Segoe UI', 'Roboto', 'Oxygen', 'Ubuntu', 'Cantarell', 'Helvetica Neue', 'Fira Sans', 'Droid Sans', 'Arial', 'sans-serif']), table_font_size=OptionsInfo(scss=True, category='table', type='px', value='16px'), table_font_weight=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_style=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_color=OptionsInfo(scss=True, category='table', type='value', value='#333333'), table_font_color_light=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_border_top_include=OptionsInfo(scss=False, category='table', type='boolean', value=True), table_border_top_style=OptionsInfo(scss=True, category='table', type='value', value='solid'), tabl

### 8. Proxies alternativos de conectividade (Caminho 2)

Testar quatro proxies além da mediana de penetração: (1) extremos Q75 vs Q25, (2) % fibra óptica, (3) tratamento contínuo (dose-resposta), (4) apenas capitais. Para cada um, rodar Triple-DiD e placebo temporal (Dez/2021). Se algum proxy passar no placebo, sugere que o problema está na definição de conectividade.

**Nota:** Rode as células 8a a 8g em sequência. Com `USE_CACHE_3B = True`, cada célula carrega só a amostra necessária do cache e libera memória ao final.

In [11]:
# Etapa 3b.8a — Preparação e gravação do cache (Proxies)
outcome = "ln_salario_real_adm"

# Garantir placebo da seção 6; se não rodou, calcula aqui
try:
    _ = m_placebo.pvalue()
except NameError:
    df["post_placebo"] = ((df["ano"] == 2021) & (df["mes"] >= 12)) | (df["ano"] > 2021)
    df["triple_did_placebo"] = df["post_placebo"].astype(int) * df["alta_exp"] * df["alta_conectividade"]
    df["post_alta_exp_placebo"] = df["post_placebo"].astype(int) * df["alta_exp"]
    m_placebo = pf.feols("ln_salario_real_adm ~ triple_did_placebo + post_alta_exp_placebo | cbo_4d + uf_periodo", data=df.loc[df[outcome].notna()], vcov=VCOV_SPEC)

# Variáveis de dose e capital
exp_col = "ilo_exposure_score" if "ilo_exposure_score" in df.columns else "alta_exp"
df["dose_triple"] = df["penetracao_bl"] * (df[exp_col] if exp_col == "ilo_exposure_score" else df[exp_col].astype(float)) * df["post"]
df["dose_exp_post"] = (df[exp_col].astype(float) if exp_col == "alta_exp" else df[exp_col]) * df["post"]
df["dose_conect_post"] = df["penetracao_bl"] * df["post"]
df["dose_exp_conect"] = (df[exp_col].astype(float) if exp_col == "alta_exp" else df[exp_col]) * df["penetracao_bl"]
if "capital" in df.columns:
    df["triple_did_capital"] = df["post"] * df["alta_exp"] * df["capital"]
    df["post_capital"] = df["post"] * df["capital"]
    df["alta_exp_capital"] = df["alta_exp"] * df["capital"]

# Amostra de regressão
print("8a: Preparando amostra de regressão…")
df_reg = df.loc[df[outcome].notna()].copy()

# Amostra extremos Q25/Q75
q25, q75 = df["penetracao_bl"].quantile(0.25), df["penetracao_bl"].quantile(0.75)
df_extremos = df[(df["penetracao_bl"] <= q25) | (df["penetracao_bl"] >= q75)].copy()
df_extremos["alta_conect_extremo"] = (df_extremos["penetracao_bl"] >= q75).astype(int)
df_extremos["triple_did_extremo"] = df_extremos["post"] * df_extremos["alta_exp"] * df_extremos["alta_conect_extremo"]
df_extremos["post_alta_conect_ext"] = df_extremos["post"] * df_extremos["alta_conect_extremo"]
df_extremos["alta_exp_alta_conect_ext"] = df_extremos["alta_exp"] * df_extremos["alta_conect_extremo"]
df_extremos["post_placebo"] = ((df_extremos["ano"] == 2021) & (df_extremos["mes"] >= 12)) | (df_extremos["ano"] > 2021)
df_extremos["triple_placebo_ext"] = df_extremos["post_placebo"].astype(int) * df_extremos["alta_exp"] * df_extremos["alta_conect_extremo"]
df_extremos["post_placebo_alta_exp"] = df_extremos["post_placebo"].astype(int) * df_extremos["alta_exp"]
df_extremos["post_placebo_alta_conect_ext"] = df_extremos["post_placebo"].astype(int) * df_extremos["alta_conect_extremo"]
df_ext = df_extremos.loc[df_extremos[outcome].notna()].copy()

if USE_CACHE_3B:
    df_reg.to_parquet(CACHE_DIR / "painel_3b_df_reg.parquet", index=False)
    df_ext.to_parquet(CACHE_DIR / "painel_3b_df_ext.parquet", index=False)
    if (CACHE_DIR / "resultados_proxy.csv").exists():
        (CACHE_DIR / "resultados_proxy.csv").unlink()
    print("Cache salvo em", CACHE_DIR.resolve())
    del df_reg, df_extremos, df_ext
    print("8a concluído. Memória liberada.")
else:
    print("8a concluído (sem cache). df_reg e df_ext em memória para 8b–8f.")

8a: Preparando amostra de regressão…
Cache salvo em /Users/manebrasil/Documents/Projects/Dissetação Mestrado/notebook/data/cache
8a concluído. Memória liberada.


In [12]:
# Etapa 3b.8b — Proxy Original (mediana)
outcome = "ln_salario_real_adm"
RESULTADOS_PROXY_CSV = CACHE_DIR / "resultados_proxy.csv"

if USE_CACHE_3B and (CACHE_DIR / "painel_3b_df_reg.parquet").exists():
    df_reg = pd.read_parquet(CACHE_DIR / "painel_3b_df_reg.parquet")
else:
    if "df_reg" not in dir():
        raise RuntimeError("Rode a célula 8a antes.")

# m_placebo: do escopo (seção 6) ou do cache
try:
    pval_placebo = m_placebo.pvalue().get("triple_did_placebo")
except NameError:
    import json
    with open(CACHE_DIR / "placebo_secao6.json") as f:
        pval_placebo = json.load(f)["triple_did_placebo"]["pval"]

formula_orig = "ln_salario_real_adm ~ triple_did + post_alta_exp + post_alta_conect + alta_exp_alta_conect | cbo_4d + uf_periodo"
m_orig = pf.feols(formula_orig, data=df_reg, vcov=VCOV_SPEC)
row = {"proxy": "Original (mediana)", "coef": m_orig.coef().get("triple_did"), "se": m_orig.se().get("triple_did"), "pval": m_orig.pvalue().get("triple_did"), "placebo_pval": pval_placebo}
pd.DataFrame([row]).to_csv(RESULTADOS_PROXY_CSV, index=False)
if USE_CACHE_3B:
    del df_reg
print("8b concluído.")

8b concluído.


In [13]:
# Etapa 3b.8c — Proxy Extremos (Q75/Q25)
outcome = "ln_salario_real_adm"

if USE_CACHE_3B and (CACHE_DIR / "painel_3b_df_ext.parquet").exists():
    df_ext = pd.read_parquet(CACHE_DIR / "painel_3b_df_ext.parquet")
else:
    if "df_ext" not in dir():
        raise RuntimeError("Rode a célula 8a antes.")

formula_ext = f"{outcome} ~ triple_did_extremo + post_alta_exp + post_alta_conect_ext + alta_exp_alta_conect_ext | cbo_4d + uf_periodo"
m_ext = pf.feols(formula_ext, data=df_ext, vcov=VCOV_SPEC)
m_placebo_ext = pf.feols(f"{outcome} ~ triple_placebo_ext + post_placebo_alta_exp + post_placebo_alta_conect_ext + alta_exp_alta_conect_ext | cbo_4d + uf_periodo", data=df_ext, vcov=VCOV_SPEC)
pval_placebo_ext = m_placebo_ext.pvalue().get("triple_placebo_ext", np.nan)
row = {"proxy": "Extremos Q75/Q25", "coef": m_ext.coef().get("triple_did_extremo"), "se": m_ext.se().get("triple_did_extremo"), "pval": m_ext.pvalue().get("triple_did_extremo"), "placebo_pval": pval_placebo_ext}
pd.DataFrame([row]).to_csv(RESULTADOS_PROXY_CSV, mode="a", header=False, index=False)
if USE_CACHE_3B:
    del df_ext
print("8c concluído.")

8c concluído.


In [14]:
# Etapa 3b.8d — Proxy % Fibra (requer painel v2)
outcome = "ln_salario_real_adm"

if USE_CACHE_3B and (CACHE_DIR / "painel_3b_df_reg.parquet").exists():
    df_reg = pd.read_parquet(CACHE_DIR / "painel_3b_df_reg.parquet")
else:
    if "df_reg" not in dir():
        raise RuntimeError("Rode a célula 8a antes.")

if "triple_did_fibra" in df_reg.columns:
    df_reg["triple_placebo_fibra"] = df_reg["post_placebo"].astype(int) * df_reg["alta_exp"] * df_reg["alta_fibra"]
    df_reg["post_placebo_alta_fibra"] = df_reg["post_placebo"].astype(int) * df_reg["alta_fibra"]
    formula_fibra = f"{outcome} ~ triple_did_fibra + post_alta_exp + post_alta_fibra + alta_exp_alta_fibra | cbo_4d + uf_periodo"
    m_fibra = pf.feols(formula_fibra, data=df_reg, vcov=VCOV_SPEC)
    m_pl_fibra = pf.feols(f"{outcome} ~ triple_placebo_fibra + post_alta_exp_placebo + post_placebo_alta_fibra + alta_exp_alta_fibra | cbo_4d + uf_periodo", data=df_reg, vcov=VCOV_SPEC)
    row = {"proxy": "% Fibra", "coef": m_fibra.coef().get("triple_did_fibra"), "se": m_fibra.se().get("triple_did_fibra"), "pval": m_fibra.pvalue().get("triple_did_fibra"), "placebo_pval": m_pl_fibra.pvalue().get("triple_placebo_fibra")}
else:
    row = {"proxy": "% Fibra", "coef": np.nan, "se": np.nan, "pval": np.nan, "placebo_pval": np.nan}
pd.DataFrame([row]).to_csv(RESULTADOS_PROXY_CSV, mode="a", header=False, index=False)
if USE_CACHE_3B:
    del df_reg
print("8d concluído.")

/opt/homebrew/lib/python3.10/site-packages/pyfixest/estimation/feols_.py:2847: UserWarning: 
            3 variables dropped due to multicollinearity.
            The following variables are dropped: ['triple_did_fibra', 'post_alta_fibra', 'alta_exp_alta_fibra'].
            
  warnings.warn(
/opt/homebrew/lib/python3.10/site-packages/pyfixest/estimation/feols_.py:2847: UserWarning: 
            3 variables dropped due to multicollinearity.
            The following variables are dropped: ['triple_placebo_fibra', 'post_placebo_alta_fibra', 'alta_exp_alta_fibra'].
            
  warnings.warn(


8d concluído.


In [15]:
# Etapa 3b.8e — Proxy Contínuo (dose-resposta)
outcome = "ln_salario_real_adm"

if USE_CACHE_3B and (CACHE_DIR / "painel_3b_df_reg.parquet").exists():
    df_reg = pd.read_parquet(CACHE_DIR / "painel_3b_df_reg.parquet")
else:
    if "df_reg" not in dir():
        raise RuntimeError("Rode a célula 8a antes.")

formula_cont = f"{outcome} ~ dose_triple + dose_exp_post + dose_conect_post + dose_exp_conect | cbo_4d + uf_periodo"
m_cont = pf.feols(formula_cont, data=df_reg, vcov=VCOV_SPEC)
row = {"proxy": "Contínuo", "coef": m_cont.coef().get("dose_triple"), "se": m_cont.se().get("dose_triple"), "pval": m_cont.pvalue().get("dose_triple"), "placebo_pval": np.nan}
pd.DataFrame([row]).to_csv(RESULTADOS_PROXY_CSV, mode="a", header=False, index=False)
if USE_CACHE_3B:
    del df_reg
print("8e concluído.")

8e concluído.


In [16]:
# Etapa 3b.8f — Proxy Capitais (requer painel v2)
outcome = "ln_salario_real_adm"

if USE_CACHE_3B and (CACHE_DIR / "painel_3b_df_reg.parquet").exists():
    df_reg = pd.read_parquet(CACHE_DIR / "painel_3b_df_reg.parquet")
else:
    if "df_reg" not in dir():
        raise RuntimeError("Rode a célula 8a antes.")

if "capital" in df_reg.columns:
    try:
        formula_cap = f"{outcome} ~ triple_did_capital + post_alta_exp + post_capital + alta_exp_capital | cbo_4d + uf_periodo"
        m_cap = pf.feols(formula_cap, data=df_reg, vcov=VCOV_SPEC)
        row = {"proxy": "Capitais", "coef": m_cap.coef().get("triple_did_capital"), "se": m_cap.se().get("triple_did_capital"), "pval": m_cap.pvalue().get("triple_did_capital"), "placebo_pval": np.nan}
    except Exception:
        row = {"proxy": "Capitais", "coef": np.nan, "se": np.nan, "pval": np.nan, "placebo_pval": np.nan}
else:
    row = {"proxy": "Capitais", "coef": np.nan, "se": np.nan, "pval": np.nan, "placebo_pval": np.nan}
pd.DataFrame([row]).to_csv(RESULTADOS_PROXY_CSV, mode="a", header=False, index=False)
if USE_CACHE_3B:
    del df_reg
print("8f concluído.")

8f concluído.


In [17]:
# Etapa 3b.8g — Montagem da tabela
RESULTADOS_PROXY_CSV = CACHE_DIR / "resultados_proxy.csv"
if not RESULTADOS_PROXY_CSV.exists():
    print("Rode as células 8b–8f antes para gerar resultados_proxy.csv")
else:
    tab_proxy = pd.read_csv(RESULTADOS_PROXY_CSV)
    tab_proxy["sig"] = tab_proxy["pval"].apply(lambda p: "***" if p < 0.01 else ("**" if p < 0.05 else ("*" if p < 0.1 else "")))
    print("Tabela comparativa — Triple-DiD por proxy de conectividade")
    print(tab_proxy.to_string())

Tabela comparativa — Triple-DiD por proxy de conectividade
                proxy      coef        se          pval  placebo_pval  sig
0  Original (mediana) -0.015759  0.008346  5.944694e-02      0.000057    *
1    Extremos Q75/Q25 -0.021561  0.007295  3.322350e-03      0.017556  ***
2             % Fibra       NaN       NaN           NaN           NaN     
3            Contínuo -0.058241  0.011591  6.506810e-07           NaN  ***
4            Capitais -0.113379  0.016465  1.345479e-11           NaN  ***


### 9. Decomposição etária do Triple-DiD

Rodar Triple-DiD para cada outcome por faixa etária (admissões e salário por jovem/intermediário/senior, share de jovens/seniores, razão salarial jovem/senior). Se a hipótese de compensação etária estiver correta: share_jovem ↓, share_senior ↑, ln_sal_real_jovem ↓, razao_sal_jovem_senior ↓.

**Cache:** Com `USE_CACHE_3B = True`, os resultados são salvos em `resultados_idade.csv`. Se o kernel cair ou você reexecutar a célula, os outcomes já calculados são carregados do cache e só os faltantes são estimados (um por vez, com menos memória).

In [18]:
# Etapa 3b.9 — Decomposição etária do Triple-DiD (com cache; reaproveita resultados anteriores)
outcomes_idade = {
    "Admissões jovens (log)": "ln_adm_jovem",
    "Admissões intermediários (log)": "ln_adm_intermediario",
    "Admissões seniores (log)": "ln_adm_senior",
    "Share jovens nas admissões": "share_jovem",
    "Share seniores nas admissões": "share_senior",
    "Salário real jovens (log)": "ln_sal_real_jovem",
    "Salário real intermediários (log)": "ln_sal_real_intermediario",
    "Salário real seniores (log)": "ln_sal_real_senior",
    "Razão salarial jovem/senior": "razao_sal_jovem_senior",
}

CACHE_IDADE_CSV = CACHE_DIR / "resultados_idade.csv"
COLS_REGR = ["triple_did", "post_alta_exp", "post_alta_conect", "alta_exp_alta_conect", "cbo_4d", "uf_periodo", "id_municipio"]
resultados_idade = {}
if CACHE_IDADE_CSV.exists() and USE_CACHE_3B:
    cache_df = pd.read_csv(CACHE_IDADE_CSV)
    for _, r in cache_df.iterrows():
        resultados_idade[r["outcome_name"]] = {"coef": r["coef"], "se": r["se"], "pval": r["pval"], "n": int(r["n"])}
    print(f"Carregados {len(resultados_idade)} resultados do cache.")

formula_base = "{outcome} ~ triple_did + post_alta_exp + post_alta_conect + alta_exp_alta_conect | cbo_4d + uf_periodo"
for nome, outcome_var in outcomes_idade.items():
    if nome in resultados_idade:
        continue
    if outcome_var not in df.columns:
        print(f"SKIP {nome}: coluna {outcome_var} não encontrada (rode 3a com painel v2).")
        continue
    # Só as colunas necessárias para a regressão (reduz memória)
    use_cols = [outcome_var] + [c for c in COLS_REGR if c in df.columns]
    mask = df[outcome_var].notna()
    df_valid = df.loc[mask, use_cols].copy()
    if len(df_valid) < 1000:
        print(f"SKIP {nome}: apenas {len(df_valid)} obs válidas")
        del df_valid
        continue
    formula = formula_base.format(outcome=outcome_var)
    try:
        model = pf.feols(formula, data=df_valid, vcov=VCOV_SPEC)
        coef = float(model.coef().get("triple_did"))
        se = float(model.se().get("triple_did"))
        pval = float(model.pvalue().get("triple_did"))
        n = len(df_valid)
        resultados_idade[nome] = {"coef": coef, "se": se, "pval": pval, "n": n}
        if USE_CACHE_3B:
            pd.DataFrame([{"outcome_name": nome, "outcome_var": outcome_var, "coef": coef, "se": se, "pval": pval, "n": n}]).to_csv(CACHE_IDADE_CSV, mode="a", header=not CACHE_IDADE_CSV.exists(), index=False)
        print(f"OK {nome}")
    except Exception as e:
        print(f"ERRO {nome}: {e}")
    del df_valid

if resultados_idade:
    df_resultados_idade = pd.DataFrame(resultados_idade).T
    df_resultados_idade["sig"] = df_resultados_idade["pval"].apply(
        lambda p: "***" if p < 0.01 else ("**" if p < 0.05 else ("*" if p < 0.1 else ""))
    )
    print("Triple-DiD por outcome etário (coef = triple_did):")
    print(df_resultados_idade.to_string())
else:
    print("Nenhum outcome etário disponível. Execute o Notebook 3a e exporte o painel v2.")

Carregados 9 resultados do cache.
Triple-DiD por outcome etário (coef = triple_did):
                                       coef        se          pval          n  sig
Admissões jovens (log)            -0.330124  0.043301  8.659740e-14  5534808.0  ***
Admissões intermediários (log)    -0.351799  0.050120  5.585532e-12  5534808.0  ***
Admissões seniores (log)          -0.182257  0.027490  7.022161e-11  5534808.0  ***
Share jovens nas admissões        -0.001168  0.005800  8.405146e-01  4404189.0     
Share seniores nas admissões      -0.001866  0.002645  4.805814e-01  4404189.0     
Salário real jovens (log)         -0.631805  0.076151  6.661338e-16  5534808.0  ***
Salário real intermediários (log) -0.534896  0.076899  8.498313e-12  5534808.0  ***
Salário real seniores (log)       -0.723624  0.116553  9.486867e-10  5534808.0  ***
Razão salarial jovem/senior       -7.271724  8.598501  3.980303e-01  1391959.0     


### 10. Modelo reformulado: impacto sobre jovens

Hipótese: após o ChatGPT, a **participação de jovens nas contratações** (share_jovem) caiu mais em ocupações expostas à IA e em municípios conectados. Outcome principal: **share_jovem** = admissões de jovens (&lt;30) / total de admissões. Três especificações (pura, com tendência, com controles demográficos), placebo temporal e event study por conectividade.

**Memória:** A célula usa apenas as colunas necessárias em cada bloco e libera os DataFrames entre as etapas para evitar travamento.

In [19]:
# Etapa 3b.10 — Modelo reformulado: share_jovem (DataFrames mínimos para reduzir memória)
if "share_jovem" not in df.columns:
    print("share_jovem não disponível. Execute o Notebook 3a e exporte o painel v2.")
else:
    # Colunas necessárias para as 3 especificações (evita cópia do painel inteiro)
    cols_j = ["share_jovem", "triple_did", "post_alta_exp", "post_alta_conect", "alta_exp_alta_conect", "alta_exp", "alta_conectividade", "cbo_4d", "uf_periodo", "id_municipio"]
    if "periodo" in df.columns:
        cols_j.append("periodo")
    for c in ["pct_superior_adm", "pct_mulher_adm", "ln_pib_pc"]:
        if c in df.columns:
            cols_j.append(c)
    cols_j = [c for c in cols_j if c in df.columns]
    df_j = df.loc[df["share_jovem"].notna(), cols_j].copy()
    if "periodo" in df_j.columns and "trend_exp_conect" not in df_j.columns:
        periodos_ordenados = sorted(df_j["periodo"].unique())
        trend_map = {p: i + 1 for i, p in enumerate(periodos_ordenados)}
        df_j["trend"] = df_j["periodo"].map(trend_map)
        df_j["trend_exp_conect"] = df_j["alta_exp"] * df_j["alta_conectividade"] * df_j["trend"]
        df_j["trend_exp"] = df_j["alta_exp"] * df_j["trend"]
        df_j["trend_conect"] = df_j["alta_conectividade"] * df_j["trend"]

    formula_j1 = "share_jovem ~ triple_did + post_alta_exp + post_alta_conect + alta_exp_alta_conect | cbo_4d + uf_periodo"
    m_jovem_1 = pf.feols(formula_j1, data=df_j, vcov=VCOV_SPEC)
    formula_j2 = "share_jovem ~ triple_did + post_alta_exp + post_alta_conect + alta_exp_alta_conect + trend_exp_conect + trend_exp + trend_conect | cbo_4d + uf_periodo"
    m_jovem_2 = pf.feols(formula_j2, data=df_j, vcov=VCOV_SPEC)
    controles = [c for c in ["pct_superior_adm", "pct_mulher_adm", "ln_pib_pc"] if c in df_j.columns]
    formula_j3 = "share_jovem ~ triple_did + post_alta_exp + post_alta_conect + alta_exp_alta_conect + trend_exp_conect + trend_exp + trend_conect + " + " + ".join(controles) + " | cbo_4d + uf_periodo" if controles else formula_j2
    m_jovem_3 = pf.feols(formula_j3, data=df_j.dropna(subset=controles) if controles else df_j, vcov=VCOV_SPEC)
    print("Três especificações — outcome: share_jovem")
    pf.etable([m_jovem_1, m_jovem_2, m_jovem_3])
    del df_j

    # Placebo: só colunas necessárias
    mask_pre = (df["ano"] < 2022) | ((df["ano"] == 2022) & (df["mes"] < 11)) if "ano" in df.columns else (df["periodo_dt"] < pd.Timestamp("2022-11-01")) if "periodo_dt" in df.columns else pd.Series(False, index=df.index)
    cols_pre = ["share_jovem", "alta_exp", "alta_conectividade", "alta_exp_alta_conect", "cbo_4d", "uf_periodo", "id_municipio"]
    if "ano" in df.columns:
        cols_pre.extend(["ano", "mes"])
    if "periodo_dt" in df.columns:
        cols_pre.append("periodo_dt")
    cols_pre = [c for c in cols_pre if c in df.columns]
    df_pre = df.loc[mask_pre, cols_pre].copy()
    df_pre = df_pre.dropna(subset=["share_jovem"])
    if "periodo_dt" not in df_pre.columns and "ano" in df_pre.columns:
        df_pre["periodo_dt"] = pd.to_datetime(df_pre["ano"].astype(str) + "-" + df_pre["mes"].astype(str).str.zfill(2) + "-01", errors="coerce")
    df_pre["post_placebo"] = (df_pre["periodo_dt"] >= pd.Timestamp("2021-12-01")).astype(int) if "periodo_dt" in df_pre.columns else ((df_pre["ano"] == 2021) & (df_pre["mes"] >= 12)) | (df_pre["ano"] > 2021)
    df_pre["triple_placebo"] = df_pre["post_placebo"].astype(int) * df_pre["alta_exp"] * df_pre["alta_conectividade"]
    df_pre["post_placebo_alta_exp"] = df_pre["post_placebo"].astype(int) * df_pre["alta_exp"]
    df_pre["post_placebo_alta_conect"] = df_pre["post_placebo"].astype(int) * df_pre["alta_conectividade"]
    if "alta_exp_alta_conect" not in df_pre.columns:
        df_pre["alta_exp_alta_conect"] = df_pre["alta_exp"] * df_pre["alta_conectividade"]
    m_placebo_j = pf.feols("share_jovem ~ triple_placebo + post_placebo_alta_exp + post_placebo_alta_conect + alta_exp_alta_conect | cbo_4d + uf_periodo", data=df_pre, vcov=VCOV_SPEC)
    print(f"Placebo share_jovem (Dez/2021): coef = {m_placebo_j.coef().get('triple_placebo', np.nan):.4f}, p = {m_placebo_j.pvalue().get('triple_placebo', np.nan):.4f}")
    del df_pre

    # Event study: só colunas necessárias; um grupo de conectividade por vez
    if "tempo_relativo_meses" in df.columns:
        BIN_MIN, BIN_MAX, REF = -12, 24, -1
        cols_es = ["share_jovem", "tempo_relativo_meses", "alta_conectividade", "alta_exp", "cbo_4d", "uf_periodo", "id_municipio"]
        cols_es = [c for c in cols_es if c in df.columns]
        df_es = df.loc[df["share_jovem"].notna(), cols_es].copy()
        df_es["t_bin"] = df_es["tempo_relativo_meses"].clip(lower=BIN_MIN, upper=BIN_MAX)
        for conn_val, label in [(1, "Alta"), (0, "Baixa")]:
            d = df_es[df_es["alta_conectividade"] == conn_val].copy()
            ts = [t for t in sorted(d["t_bin"].unique()) if t != REF]
            did_vars = [f"did_m{-t}" if t < 0 else f"did_{t}" for t in ts]
            for t, v in zip(ts, did_vars):
                d[v] = ((d["t_bin"] == t) & (d["alta_exp"] == 1)).astype(int)
            if did_vars:
                m_es = pf.feols(f"share_jovem ~ {' + '.join(did_vars)} | cbo_4d + uf_periodo", data=d, vcov=VCOV_SPEC)
                print(f"Event study share_jovem — {label} conect.: {len(ts)} coeficientes")
            del d
        del df_es
    print("3b.10 concluído.")

/opt/homebrew/lib/python3.10/site-packages/pyfixest/estimation/model_matrix_fixest_.py:215: UserWarning: 1 singleton fixed effect(s) detected. These observations are dropped from the model.
  warnings.warn(
/opt/homebrew/lib/python3.10/site-packages/pyfixest/estimation/model_matrix_fixest_.py:215: UserWarning: 1 singleton fixed effect(s) detected. These observations are dropped from the model.
  warnings.warn(
/opt/homebrew/lib/python3.10/site-packages/pyfixest/estimation/model_matrix_fixest_.py:215: UserWarning: 1 singleton fixed effect(s) detected. These observations are dropped from the model.
  warnings.warn(


Três especificações — outcome: share_jovem


/opt/homebrew/lib/python3.10/site-packages/pyfixest/estimation/model_matrix_fixest_.py:215: UserWarning: 3 singleton fixed effect(s) detected. These observations are dropped from the model.
  warnings.warn(


Placebo share_jovem (Dez/2021): coef = -0.0064, p = 0.3391


/opt/homebrew/lib/python3.10/site-packages/pyfixest/estimation/model_matrix_fixest_.py:215: UserWarning: 1 singleton fixed effect(s) detected. These observations are dropped from the model.
  warnings.warn(


Event study share_jovem — Alta conect.: 36 coeficientes


/opt/homebrew/lib/python3.10/site-packages/pyfixest/estimation/model_matrix_fixest_.py:215: UserWarning: 6 singleton fixed effect(s) detected. These observations are dropped from the model.
  warnings.warn(


Event study share_jovem — Baixa conect.: 36 coeficientes
3b.10 concluído.


### Verificação metodológica (conferência com o plano Etapa 3)

- **β₇ (triple_did):** Interpretar como a diferença do efeito da IA entre municípios de alta e baixa conectividade. β₇ &lt; 0 em salário real = efeito mais negativo onde a adoção de IA é viável.
- **Conectividade pré-tratamento:** Medida Jan–Out/2022 para evitar endogeneidade.
- **FE:** cbo_4d + uf_periodo absorvem choques estaduais; robustez com id_municipio + periodo.
- **Clustering:** Por id_municipio; robustez multiway (ocupação + município).
- **Comparação com Etapa 2:** Apresentar 3 como extensão — efeito médio concentrado nos municípios conectados.